### Библиотеки, которые необходимы для работы 
- numpy и pandas - вектора и матрицы
- nltk - основная библиотека для работы с nlp
- sklearn - библиотека для машинного обучения
- pymorphy2 - морфологический анализатор (приведение слов к нормальной форме)
- vaderSentiment - пакет для анализа тональности текста
- re - работа с регулярными выражениями
- nlpia - получение датасетов
- collections - специализированные типы данных

### Обработка текста делится на следующие этапы:
1. Токенизация (разделение на слова и предложения)
2. Приведение к нижнему регистру
3. Удаление стоп-слов, знаков препинания, числовых значений
4. Лемматизация (Приведение к нормальной форме слова)
5. Стемминг (Приведение к основе слова)
6. Векторизация

### Предварительная обработка текста

In [15]:
import nltk
import pandas as pd
import numpy as np

In [16]:
# возьмём исходный текст для анализа
corpus = 'When we were in Paris we visited a lot of museums. We first went to the Louvre, the largest art museum in the world. I have always been interested in art so I spent many hours there. The museum is enourmous, so a week there would not be enough.'

### Шаг 1. Токенизация 

#### Токенизация с помощью word_tokenize и sent_tokenize

In [17]:
# импортируем метод sent_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

# Разделим  текст на предложения
# скачиваем модель, которая будет делить на предложения
nltk.download('punkt')
print('')

# и применяем метод к нашему тексту
sentences = sent_tokenize(corpus)
sentences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FerrariBoy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['When we were in Paris we visited a lot of museums.',
 'We first went to the Louvre, the largest art museum in the world.',
 'I have always been interested in art so I spent many hours there.',
 'The museum is enourmous, so a week there would not be enough.']

In [18]:
# разобьём на слова первое предложение
print(word_tokenize(sentences[0]))

['When', 'we', 'were', 'in', 'Paris', 'we', 'visited', 'a', 'lot', 'of', 'museums', '.']


In [19]:
# теперь проделаем это со всеми предложениями

# для этого создадим пустой список
tokens = []
# в цикле for пройдемся по каждому предложению
for sentence in sentences:
    # создадим списки из токенов
    t = word_tokenize(sentence)
    # и присоединим списки друг к другу
    tokens.extend(t)

print(tokens)

['When', 'we', 'were', 'in', 'Paris', 'we', 'visited', 'a', 'lot', 'of', 'museums', '.', 'We', 'first', 'went', 'to', 'the', 'Louvre', ',', 'the', 'largest', 'art', 'museum', 'in', 'the', 'world', '.', 'I', 'have', 'always', 'been', 'interested', 'in', 'art', 'so', 'I', 'spent', 'many', 'hours', 'there', '.', 'The', 'museum', 'is', 'enourmous', ',', 'so', 'a', 'week', 'there', 'would', 'not', 'be', 'enough', '.']


#### Токенизация с помощью RegexpTokenizer или TreebankWordTokenizer

In [20]:
from nltk.tokenize import RegexpTokenizer
#from nltk.tokenize import TreebankWordTokenizer

tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+|\S+')
#tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(corpus)

['When',
 'we',
 'were',
 'in',
 'Paris',
 'we',
 'visited',
 'a',
 'lot',
 'of',
 'museums',
 '.',
 'We',
 'first',
 'went',
 'to',
 'the',
 'Louvre',
 ',',
 'the',
 'largest',
 'art',
 'museum',
 'in',
 'the',
 'world',
 '.',
 'I',
 'have',
 'always',
 'been',
 'interested',
 'in',
 'art',
 'so',
 'I',
 'spent',
 'many',
 'hours',
 'there',
 '.',
 'The',
 'museum',
 'is',
 'enourmous',
 ',',
 'so',
 'a',
 'week',
 'there',
 'would',
 'not',
 'be',
 'enough',
 '.']

#### Токенезация неформального текста из социальных сетей, таких как  Twitter  Facebook

Библиотека NLTK включает в себя токенизатор casual_tokenize для работы с короткими, неформальными, сдобренными смайликами текстами из социальных сетей, где грамматика и правописание сильно варьируются.
Функцией casual_tokenize можно выделять имена пользователей и сокращать количество повторяющихся символов в токене:

In [21]:
from nltk.tokenize.casual import casual_tokenize
message = "RT @TJMonticello Best day everrrrrrr at Monticello. Awesommmmmmeeeeeeee day :*)"
print(casual_tokenize(message))
print(casual_tokenize(message, reduce_len=True, strip_handles=True))

['RT', '@TJMonticello', 'Best', 'day', 'everrrrrrr', 'at', 'Monticello', '.', 'Awesommmmmmeeeeeeee', 'day', ':*)']
['RT', 'Best', 'day', 'everrr', 'at', 'Monticello', '.', 'Awesommmeee', 'day', ':*)']


### Шаг 2. Перевод текста в нижний регистр + Шаг 3. Удаление стоп-слов и пунктуации

 Стоп-слова — это распространенные слова на любом языке, которые встречаются очень часто, но несут в себе гораздо меньше содержательной информации о смысле фразы. Вот примеры некоторых распространенных стоп-слов:

1.    a, an;
2.   the, this;
3.   the, this;
4.   of, on.
5.   and, or;

С одной стороны, удаление стоп-слов может привести к потере смысловой составляющей текста, но с другой, увеличению словаря и используемых мощностей. Поэтому процесс фильтра стоп-слов 
зависит от конкретной ситуации.

В зависимости от используемой библиотеки готовый набор стоп-слов может отличаться.

In [11]:
# импортируем модуль стоп-слов
from nltk.corpus import stopwords

# скачаем словарь стоп-слов
nltk.download('stopwords')
print('')

# мы используем set, чтобы оставить только уникальные значения
unique_stops = set(stopwords.words('english'))
# создаём пустой список без стоп-слов
no_stops = []
# проходимся по всем токенам
for token in tokens:
    # переводим все слова в нижний регистр
    token = token.lower()
    # если токен не в списке стоп-слов и не является знаком пунктуации
    if token not in unique_stops and token.isalpha():
        # добавляем его в список
        no_stops.append(token)

print(no_stops)


['paris', 'visited', 'lot', 'museums', 'first', 'went', 'louvre', 'largest', 'art', 'museum', 'world', 'always', 'interested', 'art', 'spent', 'many', 'hours', 'museum', 'enourmous', 'week', 'would', 'enough']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FerrariBoy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Шаг 4. Лемматизация (Приведение к нормальной форме)

Примеры пакетов для лемматизации:

- Wordnet Lemmatizer

- Spacy Lemmatizer

- TextBlob

- CLiPS Pattern

- Stanford CoreNLP

- Gensim Lemmatizer

- TreeTagger

In [12]:
# импортируем класс для лемматизации
from nltk.stem import WordNetLemmatizer

# импортируем словарь
nltk.download('wordnet')
print('')
# создаём объект этого класса
lemmatizer = WordNetLemmatizer()
# и пустой список для слов после лемматизации
lemmatized = []

# проходимся по всем токенам
for token in no_stops:
    # применяем лемматизацию
    token = lemmatizer.lemmatize(token)
    # добавляем слово после лемматизации в список
    lemmatized.append(token)

print(lemmatized)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FerrariBoy\AppData\Roaming\nltk_data...



['paris', 'visited', 'lot', 'museum', 'first', 'went', 'louvre', 'largest', 'art', 'museum', 'world', 'always', 'interested', 'art', 'spent', 'many', 'hour', 'museum', 'enourmous', 'week', 'would', 'enough']


### Шаг 5. Стемминг (Приведение к основе)

Разница между лематизацией и стеммингом. 

Например, лемматизация правильно определила бы базовую форму «caring» и «care», в то время как стемминг отрезал бы «ing» и преобразовал ее в car.
### «Caring» -> Лемматизация -> «Care»

### «Caring» -> Стемминг -> «Car»

Двумя наиболее популярными алгоритмами являются стеммер Портера и Snowball. Стеммер Портера назван в честь специалиста в области компьютерных наук Мартина Портера (Martin Porter)1. Портеру мы обязаны и усовершенствованной версией его стеммера под названием Snowball2. Мартин посвятил большую часть своей долгой карьеры документированию и улучшению стеммеров ввиду важности их роли в поиске информации (поиске по ключевым словам). Описанные выше стеммеры реализуют более сложные, чем обычные регулярные выражения, правила. Это позволяет справляться со сложностями правил правописания и окончания слов английского языка:

In [22]:
# импортируем класс стеммера Porter и создаём объект этого класса
from nltk.stem import PorterStemmer
porter = PorterStemmer()

# используем list comprehension вместо цикла for для стемминга и создание нового списка
# такая запись намного короче
stemmed_p = [porter.stem(s) for s in lemmatized]
print(stemmed_p)

['pari', 'visit', 'lot', 'museum', 'first', 'went', 'louvr', 'largest', 'art', 'museum', 'world', 'alway', 'interest', 'art', 'spent', 'mani', 'hour', 'museum', 'enourm', 'week', 'would', 'enough']


### Сценарии использования

В каких случаях следует использовать лемматизатор, а в каких — стеммер? По-
следние, как правило, работают быстрее и требуют менее сложного кода и мень-
ших наборов данных. Однако они более подвержены ошибкам и сводят к одной основе гораздо большее количество слов, сокращая тем самым информационное
содержание (смысл) текста намного сильнее, чем лемматизаторы. Обе технологии уменьшают размер словаря и увеличивают неоднозначность текста, однако лем-
матизаторы работают лучше, сохраняя как можно больше полезной информации на основе применения слова в тексте и его желаемого смысла. Поэтому некоторые пакеты NLP, такие как spaCy, не включают функции для стемминга, а только ме-
тоды для лемматизации.

Если приложение связано с поиском информации, использование стемминга
и лемматизации повысит его чувствительность и сопоставит тем же словам запроса
больше документов. Тем не менее стемминг, лемматизация и выравнивание регистра
значительно снижают точность результатов поиска.

### Итог

Подводя итог сказанному выше, хочется попросить максимально избегать ис-
пользования стемминга и лемматизации, за исключением небольших текстов,
содержащих искомые слова в различном регистре. Учитывая лавинообразный
рост размеров наборов данных NLP, такое редко имеет место для документов на английском языке, разве что тексты изобилуют жаргонизмами или относятся к очень узкой области науки, техники или литературы. Впрочем,
для текстов, написанных на отличных от английского языках, лемматизация
все еще может принести пользу. Стэнфордский курс по поиску информации
целиком исключает стемминг и лемматизацию ввиду пренебрежимо малого
увеличения чувствительности и сильного снижения уровня точности

Функция для предобработки английского и русского текста (без стемминга):

In [23]:
stops_words_en = stopwords.words('english')
stops_words_ru = stopwords.words('russian')

def preprocess(text, stop_words):
    sentences = sent_tokenize(text)
    tokens = []
    for sentence in sentences:
        t = word_tokenize(sentence)
        tokens.extend(t)
    no_stops = []
    
    for token in tokens:
        token = token.lower()
        if token not in stop_words and token.isalpha():
            no_stops.append(token)
            
    lemmatizer = WordNetLemmatizer()
    lemmatized = []

    for token in no_stops:
        token = lemmatizer.lemmatize(token)
        lemmatized.append(token)
        
    return lemmatized

In [24]:
text_ru = '''Обманывая, человек прежде всего обманывает самого себя, ибо он думает, что успешно соврал, а люди поняли и из деликатности промолчали. 
Жизнь - прежде всего творчество, но это не значит, что каждый человек, чтобы жить, должен родиться художником, балериной или ученым. 
Можно творить просто добрую атмосферу вокруг себя. Человек может принести с собой атмосферу подозрительности, какого-то тягостного молчания, а может внести сразу радость, свет. 
Вот это и есть творчество.'''
preprocess(text_ru, stops_words_ru)

['обманывая',
 'человек',
 'прежде',
 'обманывает',
 'самого',
 'ибо',
 'думает',
 'успешно',
 'соврал',
 'люди',
 'поняли',
 'деликатности',
 'промолчали',
 'жизнь',
 'прежде',
 'творчество',
 'это',
 'значит',
 'каждый',
 'человек',
 'жить',
 'должен',
 'родиться',
 'художником',
 'балериной',
 'ученым',
 'творить',
 'просто',
 'добрую',
 'атмосферу',
 'вокруг',
 'человек',
 'принести',
 'собой',
 'атмосферу',
 'подозрительности',
 'тягостного',
 'молчания',
 'внести',
 'сразу',
 'радость',
 'свет',
 'это',
 'творчество']

In [25]:
text_en = '''Thomas Jefferson began building Monticello at the age of 26. 
Construction was done mostly by local masons and carpenters. 
He moved into the South Pavilion in 1770. 
Turning Monticello into a neoclassical masterpiece was Jefferson's obsession.'''
preprocess(text_en, stops_words_en)

['thomas',
 'jefferson',
 'began',
 'building',
 'monticello',
 'age',
 'construction',
 'done',
 'mostly',
 'local',
 'mason',
 'carpenter',
 'moved',
 'south',
 'pavilion',
 'turning',
 'monticello',
 'neoclassical',
 'masterpiece',
 'jefferson',
 'obsession']

# Векторизация

### Мешок слов (bag of words, BOW)

In [26]:
# из модуля collections импортируем класс Counter
from collections import Counter

# применяем класс Counter к словам после лемматизации
# на выходе нам возвращается словарь { слово : его частота в тексте }
bow_counter = Counter(lemmatized)
# print(bow_counter)

# функция most_common() упорядочивает словарь по значению
# посмотрим на первые 10 наиболее частотных слов
print(bow_counter.most_common(10))

[('museum', 3), ('art', 2), ('paris', 1), ('visited', 1), ('lot', 1), ('first', 1), ('went', 1), ('louvre', 1), ('largest', 1), ('world', 1)]


In [27]:
# импортируем класс CountVectorizer из библиотеки Scikit-learn
from sklearn.feature_extraction.text import CountVectorizer

# создаём объект этого класса и
# указываем, что хотим перевести слова в нижний регистр, а также
# отфильтровать стоп-слова через stop_words = {'english'}
vectorizer = CountVectorizer(analyzer = "word",
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = {'english'},
                             max_features = 5000)

In [28]:
# применяем этот объект к предложениям (ещё говорят документам)
bow_cv = vectorizer.fit_transform(sentences)

# на выходе получается матрица csr
print(type(bow_cv))

<class 'scipy.sparse._csr.csr_matrix'>


In [29]:
# преобразуем матрицу csr в привычный формат массива Numpy
# для этого можно использовать .toarray()
print(bow_cv.toarray())

[[0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 2 0 0 1 1 0 0]
 [0 1 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 3 0 1 0 1 0 1 0 0 1 0]
 [1 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1]]


In [30]:
# строки это предложения (документы), столбцы - слова (токены)
bow_cv.shape

(4, 34)

In [31]:
# мы можем посмотреть на используемые токены (слова)

# здесь числа это не частотность, а просто порядковый номер (индекс)
vocab = vectorizer.vocabulary_
print(vocab)

# можно вывести слова и без индекса
tokens = vectorizer.get_feature_names_out()
print(tokens)

{'when': 31, 'we': 27, 'were': 30, 'in': 9, 'paris': 20, 'visited': 26, 'lot': 13, 'of': 19, 'museums': 17, 'first': 6, 'went': 29, 'to': 25, 'the': 23, 'louvre': 14, 'largest': 12, 'art': 1, 'museum': 16, 'world': 32, 'have': 7, 'always': 0, 'been': 3, 'interested': 10, 'so': 21, 'spent': 22, 'many': 15, 'hours': 8, 'there': 24, 'is': 11, 'enourmous': 5, 'week': 28, 'would': 33, 'not': 18, 'be': 2, 'enough': 4}
['always' 'art' 'be' 'been' 'enough' 'enourmous' 'first' 'have' 'hours'
 'in' 'interested' 'is' 'largest' 'lot' 'louvre' 'many' 'museum' 'museums'
 'not' 'of' 'paris' 'so' 'spent' 'the' 'there' 'to' 'visited' 'we' 'week'
 'went' 'were' 'when' 'world' 'would']


In [32]:
# для удобства преобразуем матрицу в датафрейм
# вначале создадим индекс предложений (документов)
index_list = []

# в цикле пройдемся по элементам матрицы, обозначим их через '_'
# функция enumerate задаст каждому элементу индекс, начиная с 0
for i, _ in enumerate(bow_cv):
    # прибавим наш индекс к слову Sentence
    index_list.append(f'Sentence_{i}')

# print(index_list)

# теперь можно использовать pd.DataFrame
bow_cv_df = pd.DataFrame(data = bow_cv.toarray(),
                         index = index_list,
                         columns = tokens)
bow_cv_df

,always,art,be,been,enough,enourmous,first,have,hours,in,...,there,to,visited,we,week,went,were,when,world,would
Sentence_0,0,0,0,0,0,0,0,0,0,1,...,0,0,1,2,0,0,1,1,0,0
Sentence_1,0,1,0,0,0,0,1,0,0,1,...,0,1,0,1,0,1,0,0,1,0
Sentence_2,1,1,0,1,0,0,0,1,1,1,...,1,0,0,0,0,0,0,0,0,0
Sentence_3,0,0,1,0,1,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1


### TF-IDF
#### Способ 1. CountVectorizer + TfidfTransformer

1) Расчет TF, term frequency, частоты слов

In [34]:
# этот шаг мы уже выполнили выше
bow_cv

<4x34 sparse matrix of type '<class 'numpy.int64'>'
	with 42 stored elements in Compressed Sparse Row format>

2) Теперь нужно рассчитать IDF

In [35]:
# импортируем TfidfTransformer (CountVectorizer уже импортирован)
from sklearn.feature_extraction.text import TfidfTransformer

# создадим объект класса TfidfTransformer
tfidf_trans = TfidfTransformer(smooth_idf = True, use_idf = True)

# и рассчитаем IDF слов
tfidf_trans.fit(bow_cv)

# поместим результат в датафрейм
df_idf = pd.DataFrame(tfidf_trans.idf_, index = tokens, columns = ["idf_weights"])
#df_idf

3) TF x IDF

In [36]:
# рассчитаем TF-IDF (по сути умножим TF на IDF)
tf_idf_vector = tfidf_trans.transform(bow_cv)
tf_idf_vector

<4x34 sparse matrix of type '<class 'numpy.float64'>'
	with 42 stored elements in Compressed Sparse Row format>

In [37]:
# теперь мы можем посмотреть на показатель TF-IDF для конкретного слова в конкретном документе

# для этого переведем матрицу csr в обычный массив Numpy
df_tfidf = pd.DataFrame(tf_idf_vector.toarray(), columns = vectorizer.get_feature_names_out())

# и траспонируем его (запишем столбцы в виде строк)
print(df_tfidf.T)

                   0         1         2         3
always      0.000000  0.000000  0.328404  0.000000
art         0.000000  0.211724  0.258918  0.000000
be          0.000000  0.000000  0.000000  0.324676
been        0.000000  0.000000  0.328404  0.000000
enough      0.000000  0.000000  0.000000  0.324676
enourmous   0.000000  0.000000  0.000000  0.324676
first       0.000000  0.268544  0.000000  0.000000
have        0.000000  0.000000  0.328404  0.000000
hours       0.000000  0.000000  0.328404  0.000000
in          0.202925  0.171408  0.209616  0.000000
interested  0.000000  0.000000  0.328404  0.000000
is          0.000000  0.000000  0.000000  0.324676
largest     0.000000  0.268544  0.000000  0.000000
lot         0.317921  0.000000  0.000000  0.000000
louvre      0.000000  0.268544  0.000000  0.000000
many        0.000000  0.000000  0.328404  0.000000
museum      0.000000  0.211724  0.000000  0.255978
museums     0.317921  0.000000  0.000000  0.000000
not         0.000000  0.000000 

In [38]:
# посмотрим сколько слов оставил нам этот метод после обработки
df_tfidf.T.shape

(34, 4)

#### Способ 2. TfidfVectorizer

In [124]:
# импортируем класс TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [125]:
# создаем объект класса TfidfVectorizer
tfIdfVectorizer = TfidfVectorizer(use_idf = True, stop_words= 'english')

# сразу рассчитываем TF-IDF слов
tfIdf = tfIdfVectorizer.fit_transform(sentences)
tfIdf

<4x15 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>

In [126]:
# можно посмотреть какие слова остались после фильтрации
print(tfIdfVectorizer.get_feature_names_out())

# например по сравнению со способом 1 выпало наречие always

['art' 'enourmous' 'hours' 'interested' 'largest' 'lot' 'louvre' 'museum'
 'museums' 'paris' 'spent' 'visited' 'week' 'went' 'world']


In [127]:
# также можно посмотреть IDF слов
tfIdfVectorizer.idf_

array([1.51082562, 1.91629073, 1.91629073, 1.91629073, 1.91629073,
       1.91629073, 1.91629073, 1.51082562, 1.91629073, 1.91629073,
       1.91629073, 1.91629073, 1.91629073, 1.91629073, 1.91629073])

In [128]:
# через датафрейм мы можем связать слова и их IDF
df_idf = pd.DataFrame(tfIdfVectorizer.idf_, index = tfIdfVectorizer.get_feature_names_out(), columns = ['idf_weights'])
# df_idf.sort_values(by = 'idf_weights', ascending = False)

In [129]:
# количество предложений (документов) х количество слов
tfIdf.shape

(4, 15)

Расчет значения TF-IDF для каждого слова по каждому тексту

In [130]:
# и наконец само значение TF-IDF для конкретного слова в конкретном документе
# чем оно уникальнее для конкретного документа, тем выше показатель
df_tfidf = pd.DataFrame(tfIdf.toarray(), columns = tfIdfVectorizer.get_feature_names_out())
print(df_tfidf.T)

              0         1         2         3
art         0.0  0.344315  0.414289  0.000000
enourmous   0.0  0.000000  0.000000  0.617614
hours       0.0  0.000000  0.525473  0.000000
interested  0.0  0.000000  0.525473  0.000000
largest     0.0  0.436719  0.000000  0.000000
lot         0.5  0.000000  0.000000  0.000000
louvre      0.0  0.436719  0.000000  0.000000
museum      0.0  0.344315  0.000000  0.486934
museums     0.5  0.000000  0.000000  0.000000
paris       0.5  0.000000  0.000000  0.000000
spent       0.0  0.000000  0.525473  0.000000
visited     0.5  0.000000  0.000000  0.000000
week        0.0  0.000000  0.000000  0.617614
went        0.0  0.436719  0.000000  0.000000
world       0.0  0.436719  0.000000  0.000000


Расчет среднего значения TF-IDF для каждого слова по всем текстам

In [131]:
# рассчитаем среднее арифметическое по строкам (axis = 0)
tfIdf.mean(axis = 0)

matrix([[0.18965082, 0.15440359, 0.13136819, 0.13136819, 0.10917983,
         0.125     , 0.10917983, 0.2078122 , 0.125     , 0.125     ,
         0.13136819, 0.125     , 0.15440359, 0.10917983, 0.10917983]])

In [132]:
# преобразуем матрицу в массив Numpy
np.asarray(tfIdf.mean(axis = 0))

array([[0.18965082, 0.15440359, 0.13136819, 0.13136819, 0.10917983,
        0.125     , 0.10917983, 0.2078122 , 0.125     , 0.125     ,
        0.13136819, 0.125     , 0.15440359, 0.10917983, 0.10917983]])

In [133]:
# посмотрим сколько измерений
np.asarray(tfIdf.mean(axis = 0)).shape

(1, 15)

In [134]:
# уберем второе измерение
np.asarray(tfIdf.mean(axis = 0)).ravel()

array([0.18965082, 0.15440359, 0.13136819, 0.13136819, 0.10917983,
       0.125     , 0.10917983, 0.2078122 , 0.125     , 0.125     ,
       0.13136819, 0.125     , 0.15440359, 0.10917983, 0.10917983])

In [135]:
# снова смотрим на размерность
np.asarray(tfIdf.mean(axis = 0)).ravel().shape

(15,)

In [136]:
# преобразуем в список
mean_weights = np.asarray(tfIdf.mean(axis = 0)).ravel().tolist()
print(mean_weights)

[0.18965081782108964, 0.15440359274390048, 0.13136818731601646, 0.13136818731601646, 0.10917982746877804, 0.125, 0.10917982746877804, 0.2078121960479979, 0.125, 0.125, 0.13136818731601646, 0.125, 0.15440359274390048, 0.10917982746877804, 0.10917982746877804]


In [137]:
# создаём датафрейм из словаря
mean_weights_df = pd.DataFrame({'term': tfIdfVectorizer.get_feature_names_out(), 'mean_weights': mean_weights})

# сортируем по убыванию 10 слов с максимальным средним TF-IDF
mean_weights_df.sort_values(by = 'mean_weights', ascending = False).reset_index(drop = True).head(10)

,term,mean_weights
0,museum,0.207812
1,art,0.189651
2,enourmous,0.154404
3,week,0.154404
4,hours,0.131368
5,interested,0.131368
6,spent,0.131368
7,lot,0.125000
8,museums,0.125000
9,paris,0.125000


### Дополнительные примеры

#### Косинусное расстояние между текстовыми векторами

In [138]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from scipy.sparse.csr import csr_matrix
# import numpy as np
# import pandas as pd

In [139]:
# возьмем для простоты два текста (предложения)
text1 = 'all the world’s a stage, and all the men and women merely players'
text2 = 'you must be the change you wish to see in the world'

# объединим их в корпус
corpus = [text1, text2]

# создадим объект класса TfidfVectorizer
tfIdfVectorizer = TfidfVectorizer(use_idf = True, stop_words = 'english')

# на выходе получаем два вектора, где каждое значение - это вес (показатель tf-idf) слова
X = tfIdfVectorizer.fit_transform(corpus)

# преобразуем данные формат массива Numpy
print(X.toarray())

[[0.         0.4261596  0.4261596  0.4261596  0.4261596  0.
  0.4261596  0.30321606]
 [0.6316672  0.         0.         0.         0.         0.6316672
  0.         0.44943642]]


In [140]:
# для удобства можем посмотреть на веса в формате датафрейма
vectors_df = pd.DataFrame(data = X.toarray(),
                          index = ['vector1', 'vector2'],
                          columns = tfIdfVectorizer.get_feature_names_out())
vectors_df

,change,men,merely,players,stage,wish,women,world
vector1,0.000000,0.42616,0.42616,0.42616,0.42616,0.000000,0.42616,0.303216
vector2,0.631667,0.00000,0.00000,0.00000,0.00000,0.631667,0.00000,0.449436


Напомню формулу косинусного расстояния:

$$ \cos(\theta )={\mathbf {a} \cdot \mathbf {b} \over \|\mathbf {a} \|\|\mathbf {b} \|} $$

In [141]:
# возьмем вектора по отдельности
vector1 = X.toarray()[0]
vector2 = X.toarray()[1]

In [142]:
# вначале выполним операции в числителе формулы
numerator = np.dot(vector1, vector2)

In [143]:
# теперь займемся знаменателем и
# (1) рассчитаем длины (по большому счету, это теорема Пифагора)
vector1Len = np.linalg.norm(vector1)
vector2Len = np.linalg.norm(vector2)

# (2) перемножим их
denominator = vector1Len * vector2Len

In [144]:
# посмотрим, чему равен косинус угла между векторами
cosine = numerator/denominator
cosine

0.13627634143908643

In [145]:
# найдем угол в градусах по его косинусу
# для этого вначале вычислим угол в радианах
angle_radians = np.arccos(cosine)

# затем в градусах
angle_degrees = angle_radians * 360/2/np.pi
round(angle_degrees, 2)

82.17

#### Кластерный анализ текста

In [146]:
# в тексте ниже две темы: наука о данных и Большой театр (источник: Википедия)
text = '''
Data science is an interdisciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from noisy, structured and unstructured data.
It applies knowledge and actionable insights from data across a broad range of application domains.
Data science is related to data mining, machine learning and big data.
The Bolshoi Theatre is a historic theatre in Moscow, Russia.
It was originally designed by architect Joseph Bové, which holds ballet and opera performances.
Before the October Revolution it was a part of the Imperial Theatres of the Russian Empire along with Maly Theatre in Moscow and a few theatres in Saint Petersburg.
Data science is a concept to unify statistics, data analysis, informatics, and their related methods in order to understand and analyze actual phenomena with data.
However, data science is different from computer science and information science.
The main building of the theatre, rebuilt and renovated several times during its history, is a landmark of Moscow and Russia.
On 28 October 2011, the Bolshoi re-opened after an extensive six-year renovation.
'''

In [147]:
# создадим список из предложений
corpus = []

# для этого в цикле for пройдемся по тексту, разделив его по символу новой строки \n
for line in text.split('\n'):

  # если строка не пустая (т.е. True)
  if line:

    # переводим ее в нижний регистр
    line = line.lower()
    # и добавляем в список
    corpus.append(line)

In [148]:
corpus

['data science is an interdisciplinary field that uses scientific methods, processes, algorithms and systems to extract knowledge and insights from noisy, structured and unstructured data.',
 'it applies knowledge and actionable insights from data across a broad range of application domains.',
 'data science is related to data mining, machine learning and big data.',
 'the bolshoi theatre is a historic theatre in moscow, russia.',
 'it was originally designed by architect joseph bové, which holds ballet and opera performances.',
 'before the october revolution it was a part of the imperial theatres of the russian empire along with maly theatre in moscow and a few theatres in saint petersburg.',
 'data science is a concept to unify statistics, data analysis, informatics, and their related methods in order to understand and analyze actual phenomena with data.',
 'however, data science is different from computer science and information science.',
 'the main building of the theatre, rebuil

In [149]:
# применим TfidfVectorizer
tfIdfVectorizer = TfidfVectorizer(use_idf = True, stop_words= 'english')

# на выходе получаем векторы предложений
X = tfIdfVectorizer.fit_transform(corpus)
# print(X.toarray())

In [150]:
# импортируем алгоритм k-средних из библиотеки sklearn
from sklearn.cluster import KMeans

# так как мы знаем, что темы две, используем гиперпараметр k = 2
kmeans = KMeans(n_clusters = 2).fit(X)

In [151]:
# возьмем новые предложения, одно из области Data Science и два про Большой театр
prediction = ['Many statisticians, including Nate Silver, have argued that data science is not a new field, but rather another name for statistics.',
              'Urusov set up the theatre in collaboration with English tightrope walker Michael Maddox.',
              'Until the mid-1990s, most foreign operas were sung in Russian, but Italian and other languages have been heard more frequently on the Bolshoi stage in recent years.']

# применим две модели, сначала создадим векторы новых предложений (tfIdfVectorizer.transform),
# затем отнесем их к одному из кластеров (kmeans.predict)
kmeans.predict(tfIdfVectorizer.transform(prediction))

array([0, 1, 1])

## Плотные векторные представления слов
## Word2Vec
### Предварительно обученные векторы Word2Vec для английского и русского языков

In [39]:
import gensim.downloader

word2vec_eng = gensim.downloader.load('word2vec-google-news-300')
word2vec_rus = gensim.downloader.load('word2vec-ruscorpora-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded
[==================================================] 100.0% 198.8/198.8MB downloaded


In [40]:
# посмотрим векторное представление для слова test
word2vec_eng['test']

array([-1.42578125e-01, -3.68652344e-02,  1.35742188e-01, -6.20117188e-02,
        7.95898438e-02,  1.90429688e-02, -8.15429688e-02, -1.27929688e-01,
       -2.95410156e-02,  2.36328125e-01, -1.21582031e-01, -2.14843750e-01,
        1.29882812e-01, -2.70996094e-02, -5.20019531e-02,  2.15820312e-01,
       -1.81640625e-01,  5.10253906e-02, -1.60156250e-01, -1.76757812e-01,
        1.83105469e-02, -4.12597656e-02, -2.32421875e-01, -1.03149414e-02,
        1.45507812e-01,  5.24902344e-02, -3.96484375e-01, -1.92871094e-02,
        2.51770020e-03, -1.26953125e-02, -4.39453125e-02,  3.07617188e-02,
        9.57031250e-02, -1.75781250e-01,  1.04370117e-02,  1.89453125e-01,
       -2.36328125e-01,  4.37011719e-02,  2.81250000e-01, -2.07519531e-02,
       -1.81640625e-01, -2.17773438e-01,  2.33398438e-01,  5.29785156e-02,
       -1.13769531e-01,  9.39941406e-03, -1.49414062e-01,  1.99218750e-01,
       -1.75781250e-01,  3.16406250e-01,  8.10546875e-02, -6.12792969e-02,
       -1.52343750e-01, -

In [41]:
# какие слова есть в русском словаре 
for index, word in enumerate(word2vec_rus.index_to_key ):
    if index == 20:
        break
    print(f"word #{index}/{len(word2vec_rus.index_to_key )} is {word}")

word #0/184973 is весь_DET
word #1/184973 is человек_NOUN
word #2/184973 is мочь_VERB
word #3/184973 is год_NOUN
word #4/184973 is сказать_VERB
word #5/184973 is время_NOUN
word #6/184973 is говорить_VERB
word #7/184973 is становиться_VERB
word #8/184973 is знать_VERB
word #9/184973 is самый_DET
word #10/184973 is дело_NOUN
word #11/184973 is день_NOUN
word #12/184973 is жизнь_NOUN
word #13/184973 is рука_NOUN
word #14/184973 is очень_ADV
word #15/184973 is первый_ADJ
word #16/184973 is давать_VERB
word #17/184973 is новый_ADJ
word #18/184973 is слово_NOUN
word #19/184973 is иметь_VERB


In [42]:
# векторное представление для слова печь (глагол)
word2vec_rus['печь_VERB']

array([ 2.18535755e-02, -1.36514874e-02,  6.48699515e-03,  4.94286232e-02,
        4.72369529e-02,  2.99715698e-02,  1.84931476e-02,  5.63697964e-02,
        9.00735930e-02,  4.98076454e-02, -1.42216729e-02, -1.19277993e-02,
        4.64369878e-02, -2.20510038e-03, -5.23579530e-02,  6.86843460e-03,
        2.47351658e-02, -2.94565707e-02, -3.43381912e-02, -6.68611825e-02,
        2.16565677e-03, -3.44541185e-02,  5.10950834e-02,  3.39556672e-02,
       -1.03159972e-01,  1.10369720e-01, -4.85999370e-03, -9.61249918e-02,
       -4.24017571e-02, -6.25817403e-02,  7.29365274e-02, -3.05048935e-02,
       -1.22691244e-01,  5.59408627e-02, -3.77330072e-02, -5.60487658e-02,
        2.35838126e-02, -9.34875384e-03, -1.72916036e-02, -1.21203892e-01,
        5.48743568e-02,  4.17090692e-02,  9.80748981e-02, -1.06346466e-01,
       -2.49845888e-02, -9.09208506e-02,  7.08984062e-02,  2.82790884e-02,
        5.33705652e-02,  1.25205163e-02,  1.88979451e-02,  1.18532305e-04,
        5.06328084e-02,  

In [43]:
# векторное представление для слова печь (существительное)
word2vec_rus['печь_NOUN']

array([ 9.04111098e-03,  3.17178331e-02,  5.26587218e-02, -2.81463489e-02,
        3.36806215e-02, -1.19832382e-02,  6.69362992e-02,  5.34342714e-02,
        7.75137171e-02,  5.08050360e-02,  3.30233364e-03, -2.16485690e-02,
        5.27435094e-02,  1.38199255e-02, -7.34620243e-02, -2.33334582e-02,
        3.33712399e-02,  3.48375104e-02,  3.63502614e-02, -4.44506891e-02,
        1.17149157e-02, -4.02530469e-02, -6.20140284e-02,  7.47649819e-02,
       -3.94869968e-02, -4.06278297e-02, -8.34002271e-02, -6.80526569e-02,
       -1.62618563e-01, -5.65643683e-02,  1.65630854e-03, -1.48861082e-02,
       -3.62066291e-02,  3.73510309e-02, -2.62216832e-02, -3.96667421e-02,
       -9.02666152e-03,  1.10610299e-01, -1.12228990e-01, -1.61341488e-01,
        1.11707663e-02, -6.39451202e-03,  5.29266559e-02, -1.17891423e-01,
       -8.77382420e-03, -2.01879211e-05,  9.10191685e-02,  2.39265133e-02,
        7.54997283e-02, -1.52666410e-02,  5.74746467e-02, -5.05433492e-02,
        2.91562006e-02, -

Также мы можем найти наиболее похожие слова

In [44]:
word2vec_rus.most_similar(['печь_NOUN'])

[('печка_NOUN', 0.8358493447303772),
 ('подтопок_NOUN', 0.7017349600791931),
 ('печной_ADJ', 0.6326410174369812),
 ('полати_NOUN', 0.6188919544219971),
 ('топиться_VERB', 0.6180375814437866),
 ('горнушка_NOUN', 0.6172196865081787),
 ('топиться::по-черному_VERB', 0.6171698570251465),
 ('истапливать_VERB', 0.6131374835968018),
 ('топка_NOUN', 0.6026756763458252),
 ('печурка_NOUN', 0.6010895371437073)]

In [45]:
word2vec_rus.most_similar(['печь_VERB'])

[('напекать_VERB', 0.6473735570907593),
 ('испечь_VERB', 0.6360713243484497),
 ('жарить_VERB', 0.629960834980011),
 ('нардек_NOUN', 0.5905025005340576),
 ('выпекать_VERB', 0.5834865570068359),
 ('яшный_ADJ', 0.5677016377449036),
 ('варить_VERB', 0.5661610960960388),
 ('пирог_NOUN', 0.5652977228164673),
 ('тандыр_NOUN', 0.5573609471321106),
 ('месить::тесто_VERB', 0.5557463765144348)]

### Предобученные моедли доступные в gensim

In [46]:
list(gensim.downloader.info()['models'].keys())

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

## fastText

In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (1003/1003), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 3930 (delta 912), reused 861 (delta 861), pack-reused 2927
Receiving objects: 100% (3930/3930), 8.24 MiB | 15.24 MiB/s, done.
Resolving deltas: 100% (2506/2506), done.


In [ ]:
!cd fastText; pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/fastText
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-linux_x86_64.whl size=4381799 sha256=b1e5434f111b8dbf915cac6a8d454ba935f83234c7be9a178edd5da3e00a8a88
  Stored in directory: /tmp/pip-ephem-wheel-cache-5uwdlvj8/wheels/2d/3b/6c/b1dab8ae56dbff3fc7c26103ce1f0646f1a39f6a06db46db46
Successfully built fasttext


In [ ]:
import fasttext.util

Загрузка предварительно обученных векторных представлений fastText для русского языка

In [ ]:
fasttext.util.download_model('ru', if_exists='ignore')

'cc.ru.300.bin'

In [ ]:
ft = fasttext.load_model('cc.ru.300.bin')

Показываем векторы для слов

In [ ]:
ft.get_word_vector('печь')

array([-1.58141226e-01,  1.40892416e-01, -1.16081268e-01, -3.65050286e-02,
        1.24294966e-01,  3.56620317e-03,  1.00102514e-01, -5.51618822e-03,
       -1.54162765e-01, -2.05441654e-01, -1.23218819e-01,  7.57891387e-02,
        3.42986137e-02, -4.14514728e-02, -4.02112976e-02,  2.98824906e-02,
       -5.66644147e-02,  8.71236175e-02,  4.90642563e-02,  1.29385591e-01,
        1.04177101e-02, -1.39765948e-01,  1.96572430e-02,  9.35998783e-02,
        2.53893584e-02, -8.46895576e-02, -2.33695209e-01, -9.06276107e-02,
       -3.56095424e-03,  1.33594170e-01,  1.00002609e-01, -1.07181132e-01,
        4.29465696e-02,  5.93492389e-03, -7.90850967e-02,  3.64862420e-02,
        1.04223136e-02, -1.24680214e-01, -1.95677444e-01, -5.12315333e-02,
        1.05863772e-01, -2.60896739e-02,  1.22163713e-01,  5.37529886e-02,
        4.78603020e-02,  1.70780927e-01,  2.17139110e-01, -1.11144893e-01,
        8.68928060e-02,  1.47899717e-01,  1.56384453e-01,  1.39447808e-01,
       -4.18573767e-02,  

In [ ]:
ft.get_word_vector('печью')

array([-0.1753038 ,  0.00724031, -0.13455375, -0.00027802,  0.01874303,
       -0.01056537,  0.05781734,  0.00851268, -0.12040708, -0.16586418,
       -0.10949906,  0.08429632, -0.03035829, -0.1301005 , -0.1000965 ,
        0.02976374, -0.10448169,  0.14156461,  0.01680489,  0.07560266,
       -0.01589312, -0.09254004,  0.03390333,  0.01686832,  0.06942455,
       -0.05103813, -0.13111839, -0.14413178,  0.0188932 ,  0.12781055,
       -0.00654911, -0.04281282,  0.0656225 ,  0.05542463,  0.0549427 ,
        0.08931478,  0.06213765, -0.09079059, -0.03287039,  0.0982732 ,
        0.07428933, -0.11164604,  0.05210447,  0.01079965, -0.05714549,
        0.06393512,  0.15666708,  0.01019257,  0.14045039,  0.00819319,
        0.05840708, -0.00672886,  0.06866424,  0.00952169,  0.01235682,
        0.03092333, -0.00548269,  0.0124651 , -0.04128526,  0.04060575,
        0.02289767, -0.08650371, -0.09751538, -0.00596499, -0.025728  ,
       -0.01933727, -0.05493296,  0.07499436,  0.05560207, -0.07

In [ ]:
ft.get_nearest_neighbors('печью')

[(0.7612155675888062, 'печкой'),
 (0.7319516539573669, 'печи'),
 (0.7223564982414246, 'электропечью'),
 (0.715794026851654, 'каменкой'),
 (0.693145751953125, 'печами'),
 (0.6750300526618958, 'лежанкой'),
 (0.6694084405899048, 'плитой'),
 (0.6690734028816223, 'топкой'),
 (0.6670330762863159, 'буржуйкой'),
 (0.6548987627029419, 'парилкой')]

## GloVe

Предобученную модель GloVe можно взять из той же библиотеки gensim

In [51]:
glove_eng = gensim.downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [53]:
glove_eng['car']

array([ 0.47685 , -0.084552,  1.4641  ,  0.047017,  0.14686 ,  0.5082  ,
       -1.2228  , -0.22607 ,  0.19306 , -0.29756 ,  0.20599 , -0.71284 ,
       -1.6288  ,  0.17096 ,  0.74797 , -0.061943, -0.65766 ,  1.3786  ,
       -0.68043 , -1.7551  ,  0.58319 ,  0.25157 , -1.2114  ,  0.81343 ,
        0.094825, -1.6819  , -0.64498 ,  0.6322  ,  1.1211  ,  0.16112 ,
        2.5379  ,  0.24852 , -0.26816 ,  0.32818 ,  1.2916  ,  0.23548 ,
        0.61465 , -0.1344  , -0.13237 ,  0.27398 , -0.11821 ,  0.1354  ,
        0.074306, -0.61951 ,  0.45472 , -0.30318 , -0.21883 , -0.56054 ,
        1.1177  , -0.36595 ], dtype=float32)

In [54]:
glove_eng.most_similar(['car'])

[('truck', 0.92085862159729),
 ('cars', 0.8870189785957336),
 ('vehicle', 0.8833683729171753),
 ('driver', 0.8464019298553467),
 ('driving', 0.8384189009666443),
 ('bus', 0.8210511803627014),
 ('vehicles', 0.8174992799758911),
 ('parked', 0.7902189493179321),
 ('motorcycle', 0.7866503000259399),
 ('taxi', 0.7833929657936096)]

### Полезные ссылки на предобученные модели

1. [Библиотека Gensim](https://radimrehurek.com/gensim/index.html).
2. [Word2Vec в Gensim](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html).
3. [Библиотека fastText](https://fasttext.cc/).
4. [Предварительно обученные векторные представления для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html).
3. [Библиотека navec](https://natasha.github.io/navec/).
4. [Navec — компактные эмбеддинги для русского языка](https://natasha.github.io/navec/).

# Обучение Word2Vec на своих данных

In [59]:
import wget
url = 'https://www.dropbox.com/s/a9r0b2yj3vqvi13/banks.csv?dl=1'
filename = wget.download(url)

In [60]:
banks = pd.read_csv(filename, sep='\t', index_col='idx')

In [61]:
banks

,Score,Text
idx,,
0,Positive,В Альфа-Банке работает замечательная девушка -...
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...
2,Positive,Очень порадовала оперативность работы в банке....
3,Negative,Имела неосторожность оформить потреб. кредит в...
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...
...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...


In [63]:
banks['Preprocessed_texts'] = banks.apply(lambda row: preprocess(row['Text'], stops_words_ru), axis=1)

In [64]:
banks

,Score,Text,Preprocessed_texts
idx,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[работает, замечательная, девушка, ильясова, о..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформляя, рассрочку, видео, меге, тёплый, ста..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовала, оперативность, работы, бан..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[имела, неосторожность, оформить, потреб, кред..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшая, предыстория, нашел, сайте, mdm, ба..."
...,...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро...","[высокой, надёжности, мкб, порядочности, добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаюсь, офисе, чернореченской, физ, лиц..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...,"[попала, сегодня, очень, неприятную, ситуацию,..."


In [65]:
import gensim.models

model = gensim.models.Word2Vec(sentences=banks['Preprocessed_texts'],
                               min_count=5,
                               vector_size=100)

In [67]:
model.wv['работать']

array([ 0.08531082, -0.41040078, -1.2961476 , -0.6656613 ,  0.13305381,
        0.05057921,  0.82577777,  1.2180654 , -0.91023386,  0.10134134,
       -0.33615366, -0.27407005, -0.6530234 , -1.4890761 , -0.8337082 ,
        0.57433915,  0.6194244 , -0.8240212 ,  0.12153419, -0.69451576,
       -0.17902806,  1.0278157 ,  0.6737105 ,  0.23140655,  0.54348725,
       -1.9284692 ,  1.2045925 , -1.1909448 ,  0.21313716, -1.1008369 ,
        0.4908453 , -0.36450425,  0.2702667 ,  0.50219333, -0.6121471 ,
        0.78456193,  0.5670301 , -1.387147  , -0.46688375,  0.7368093 ,
        0.8076721 , -1.2398353 , -0.54702705,  0.94673264,  0.2986714 ,
        1.127015  , -0.40733224,  0.5913734 , -0.14767535,  0.6048299 ,
        0.7165103 , -1.798116  ,  0.49813175,  0.97189826, -1.8800792 ,
        0.20650798,  1.0045443 ,  0.48688737, -1.3046933 ,  0.3960795 ,
        0.10540345,  0.80151385, -0.2935625 , -1.937169  ,  0.206786  ,
       -1.0573828 ,  1.4519833 ,  1.8616426 ,  0.43912435, -0.55

In [68]:
model.wv['замечательный']

array([-0.14243744, -0.09188487,  0.06645581, -0.1370332 , -0.23854269,
       -0.27082646, -0.17151079,  0.483381  , -0.63861114,  0.12301378,
        0.13728085, -0.04301038,  0.30721578,  0.124957  ,  0.29704887,
       -0.13535517,  0.3045485 , -0.43771914, -0.0235233 , -0.48804736,
        0.17106162,  0.18449074,  0.2955115 , -0.16800134,  0.07400453,
        0.10816153, -0.11680923, -0.03580258, -0.4706628 , -0.10558249,
        0.3104722 , -0.2283011 ,  0.24349596, -0.30175582, -0.35549217,
        0.27378955, -0.01867673, -0.25607845,  0.1328438 , -0.2965627 ,
        0.13483457, -0.42717505, -0.48364583, -0.0091152 ,  0.2629256 ,
        0.08242698, -0.19713436, -0.0475959 , -0.08383262,  0.4330561 ,
       -0.1056499 , -0.52020514, -0.08197866,  0.00891048, -0.15326926,
        0.4785084 ,  0.3874909 , -0.19257371, -0.3508786 ,  0.29572162,
       -0.21896039,  0.06516223, -0.3863423 , -0.01128248,  0.13218783,
       -0.07674706,  0.05120785,  0.325392  , -0.2561403 , -0.13

In [69]:
model.wv.most_similar('замечательный')

[('современный', 0.9023762345314026),
 ('крупный', 0.8954980969429016),
 ('известный', 0.8729244470596313),
 ('надежный', 0.8666946887969971),
 ('достойный', 0.8656669855117798),
 ('великолепный', 0.8606410026550293),
 ('попался', 0.8525636792182922),
 ('адекватный', 0.8440741300582886),
 ('позитивный', 0.843691349029541),
 ('рядовой', 0.842851996421814)]

## Сохранение обученной модели

In [72]:
model.save('word2vec-banki.ru-50')

## Загрузка сохраненной модели

In [74]:
new_model = gensim.models.Word2Vec.load('word2vec-banki.ru-50')

In [75]:
new_model.wv['работать']

array([ 0.08531082, -0.41040078, -1.2961476 , -0.6656613 ,  0.13305381,
        0.05057921,  0.82577777,  1.2180654 , -0.91023386,  0.10134134,
       -0.33615366, -0.27407005, -0.6530234 , -1.4890761 , -0.8337082 ,
        0.57433915,  0.6194244 , -0.8240212 ,  0.12153419, -0.69451576,
       -0.17902806,  1.0278157 ,  0.6737105 ,  0.23140655,  0.54348725,
       -1.9284692 ,  1.2045925 , -1.1909448 ,  0.21313716, -1.1008369 ,
        0.4908453 , -0.36450425,  0.2702667 ,  0.50219333, -0.6121471 ,
        0.78456193,  0.5670301 , -1.387147  , -0.46688375,  0.7368093 ,
        0.8076721 , -1.2398353 , -0.54702705,  0.94673264,  0.2986714 ,
        1.127015  , -0.40733224,  0.5913734 , -0.14767535,  0.6048299 ,
        0.7165103 , -1.798116  ,  0.49813175,  0.97189826, -1.8800792 ,
        0.20650798,  1.0045443 ,  0.48688737, -1.3046933 ,  0.3960795 ,
        0.10540345,  0.80151385, -0.2935625 , -1.937169  ,  0.206786  ,
       -1.0573828 ,  1.4519833 ,  1.8616426 ,  0.43912435, -0.55

# Определение тональности текста

In [82]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

max_words = 10000
random_state = 42

In [77]:
banks

,Score,Text,Preprocessed_texts
idx,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[работает, замечательная, девушка, ильясова, о..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформляя, рассрочку, видео, меге, тёплый, ста..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовала, оперативность, работы, бан..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[имела, неосторожность, оформить, потреб, кред..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшая, предыстория, нашел, сайте, mdm, ба..."
...,...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро...","[высокой, надёжности, мкб, порядочности, добро..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаюсь, офисе, чернореченской, физ, лиц..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...,"[попала, сегодня, очень, неприятную, ситуацию,..."


Считаем частоту слов во всех отзывах

In [78]:
words = Counter()

In [79]:
for txt in banks['Preprocessed_texts']:
    words.update(txt)

Создаем словарь, упорядоченный по частоте

В словаре будем использовать 2 специальных кода:
- Код заполнитель: 0
- Неизвестное слово: 1

Нумерация слов в словаре начинается с 2.

In [80]:
# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()

Создаем словари

In [83]:
for i, word in enumerate(words.most_common(max_words - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

Функция для преобразования списка слов в список кодов

In [84]:
def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

Преобразуем все тексты в последовательность кодов слов

In [85]:
banks['Sequences'] = banks.apply(lambda row: text_to_sequence(row['Preprocessed_texts'], word_to_index), axis=1)

In [86]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,Positive,В Альфа-Банке работает замечательная девушка -...,"[работает, замечательная, девушка, ильясова, о...","[113, 7310, 57, 3119, 226, 7612, 117, 3564, 29..."
1,Negative,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформляя, рассрочку, видео, меге, тёплый, ста...","[7796, 1359, 4406, 454, 542, 543, 371, 85, 352..."
2,Positive,Очень порадовала оперативность работы в банке....,"[очень, порадовала, оперативность, работы, бан...","[7, 4525, 1090, 92, 12, 1472, 152, 6, 499, 195..."
3,Negative,Имела неосторожность оформить потреб. кредит в...,"[имела, неосторожность, оформить, потреб, кред...","[2286, 7797, 194, 3566, 9, 7, 4051, 6, 494, 30..."
4,Negative,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшая, предыстория, нашел, сайте, mdm, ба...","[1767, 7150, 1235, 72, 2, 6, 7798, 3, 858, 78,..."
...,...,...,...,...
13994,Positive,"О высокой надёжности МКБ, порядочности и добро...","[высокой, надёжности, мкб, порядочности, добро...","[3671, 706, 48, 4192, 349, 4608, 413, 24, 918,..."
13995,Positive,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаюсь, офисе, чернореченской, физ, лиц...","[876, 106, 2529, 864, 116, 106, 669, 968, 98, ..."
13996,Positive,Попала сегодня в очень неприятную ситуацию. Ре...,"[попала, сегодня, очень, неприятную, ситуацию,...","[2204, 47, 7, 8156, 188, 279, 372, 403, 50, 16..."


## Готовим данные для обучения

Преобразуем текстовые метки классов в числовые

In [87]:
mapping = {'Negative': 0, 'Positive': 1}

In [88]:
banks.replace({'Score': mapping}, inplace=True)

In [89]:
banks

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
0,1,В Альфа-Банке работает замечательная девушка -...,"[работает, замечательная, девушка, ильясова, о...","[113, 7310, 57, 3119, 226, 7612, 117, 3564, 29..."
1,0,Оформляя рассрочку в м. Видео в меге тёплый ст...,"[оформляя, рассрочку, видео, меге, тёплый, ста...","[7796, 1359, 4406, 454, 542, 543, 371, 85, 352..."
2,1,Очень порадовала оперативность работы в банке....,"[очень, порадовала, оперативность, работы, бан...","[7, 4525, 1090, 92, 12, 1472, 152, 6, 499, 195..."
3,0,Имела неосторожность оформить потреб. кредит в...,"[имела, неосторожность, оформить, потреб, кред...","[2286, 7797, 194, 3566, 9, 7, 4051, 6, 494, 30..."
4,0,Небольшая предыстория: Нашел на сайте MDM банк...,"[небольшая, предыстория, нашел, сайте, mdm, ба...","[1767, 7150, 1235, 72, 2, 6, 7798, 3, 858, 78,..."
...,...,...,...,...
13994,1,"О высокой надёжности МКБ, порядочности и добро...","[высокой, надёжности, мкб, порядочности, добро...","[3671, 706, 48, 4192, 349, 4608, 413, 24, 918,..."
13995,1,"Обслуживаюсь в офисе на Чернореченской 42а, ка...","[обслуживаюсь, офисе, чернореченской, физ, лиц...","[876, 106, 2529, 864, 116, 106, 669, 968, 98, ..."
13996,1,Попала сегодня в очень неприятную ситуацию. Ре...,"[попала, сегодня, очень, неприятную, ситуацию,...","[2204, 47, 7, 8156, 188, 279, 372, 403, 50, 16..."


### Выделяем данные для обучения и тестирования

In [90]:
train, test = train_test_split(banks, test_size=0.2)

In [91]:
train

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
3363,0,Всем посетителям Банки.ру добрый день. 05.11.2...,"[всем, посетителям, добрый, день, заключил, до...","[95, 9206, 148, 10, 5803, 66, 949, 24, 23, 460..."
4365,0,27 января 2017 я снимала деньги с карты Тинько...,"[января, снимала, деньги, карты, тинькофф, бан...","[523, 5188, 5, 8, 1489, 2, 649, 1352, 186, 261..."
259,1,В октябре 2011 года через вирусную программу с...,"[октябре, года, вирусную, программу, моего, сч...","[1962, 15, 2149, 79, 21, 332, 506, 3343, 23, 1..."
12038,0,В связи с утерей был вынужден перевыпустить ка...,"[связи, утерей, вынужден, перевыпустить, карту...","[163, 1503, 1728, 6, 13, 671, 3406, 586, 3802,..."
3784,0,ПРЕТЕНЗИЯ08.04.2015 г. в магазине торговой сет...,"[магазине, торговой, сети, ооо, расположенного...","[579, 3782, 2221, 724, 5430, 261, 2567, 1635, ..."
...,...,...,...,...
7797,1,"Я не сотрудник Альфа-банка, но скажу пару слов...","[сотрудник, скажу, пару, слов, поддержку, тех,...","[35, 965, 207, 815, 1010, 385, 1402, 1487, 428..."
6937,1,Хотелось бы поблагодарить за отличную консульт...,"[хотелось, поблагодарить, отличную, консультац...","[231, 780, 2913, 1782, 154, 7161, 5279, 7, 178..."
5824,1,"Доброго времени суток. Сегодня, (а точнее уже ...","[доброго, времени, суток, сегодня, точнее, вче...","[1215, 74, 873, 47, 1609, 394, 621, 1052, 5851..."


In [92]:
test

,Score,Text,Preprocessed_texts,Sequences
idx,,,,
7292,0,21 августа получил sms от Банка Альфа-Банк пре...,"[августа, получил, sm, банка, предлагает, дебе...","[529, 69, 1138, 2, 1622, 616, 6, 1282, 1184, 5..."
342,0,Являюсь клиентом Альфа банка с 2002 года и ник...,"[являюсь, клиентом, альфа, банка, года, возник...","[99, 53, 88, 2, 15, 1536, 97, 2040, 63, 303, 2..."
2216,0,"Добрый вечер.Не получили вознаграждение 2500,0...","[добрый, получили, вознаграждение, рублей, кре...","[148, 1080, 8545, 23, 87, 50, 2, 202, 1801, 24..."
2606,0,29 июня 2015 г. совершил на свою карту Кукуруз...,"[июня, совершил, карту, кукуруза, перевода, пр...","[555, 2520, 6, 631, 1618, 8, 776, 2, 559, 417,..."
6572,0,Добрый день!Надеюсь кто то из руководства банк...,"[добрый, день, надеюсь, руководства, банка, эт...","[148, 10, 246, 1889, 2, 4, 209, 4565, 1592, 83..."
...,...,...,...,...
7754,1,По кредитной карточке жены Visa Ситибанка 7 ию...,"[кредитной, карточке, жены, visa, ситибанка, и...","[87, 3460, 2063, 499, 2018, 555, 4086, 1189, 3..."
7023,0,Как я понимаю не я один попался на крючок по п...,"[понимаю, попался, крючок, поводу, года, зашел...","[289, 7086, 317, 15, 838, 420, 2327, 2503, 615..."
668,0,Подавал документы на кредитную карту Visa Clas...,"[подавал, документы, кредитную, карту, visa, c...","[2052, 54, 138, 6, 499, 1952, 882, 7695, 1082,..."


### Разделяем метки классов и данные для обучения

Данные для обучения

In [93]:
x_train_seq = train['Sequences']
y_train = train['Score']

In [94]:
x_train_seq

idx
3363     [95, 9206, 148, 10, 5803, 66, 949, 24, 23, 460...
4365     [523, 5188, 5, 8, 1489, 2, 649, 1352, 186, 261...
259      [1962, 15, 2149, 79, 21, 332, 506, 3343, 23, 1...
12038    [163, 1503, 1728, 6, 13, 671, 3406, 586, 3802,...
3784     [579, 3782, 2221, 724, 5430, 261, 2567, 1635, ...
                               ...                        
7797     [35, 965, 207, 815, 1010, 385, 1402, 1487, 428...
6937     [231, 780, 2913, 1782, 154, 7161, 5279, 7, 178...
5824     [1215, 74, 873, 47, 1609, 394, 621, 1052, 5851...
4858     [1514, 259, 726, 283, 158, 683, 12, 47, 1108, ...
2394     [503, 157, 6027, 876, 29, 9006, 5581, 1867, 78...
Name: Sequences, Length: 11199, dtype: object

In [95]:
y_train

idx
3363     0
4365     0
259      1
12038    0
3784     0
        ..
7797     1
6937     1
5824     1
4858     0
2394     1
Name: Score, Length: 11199, dtype: int64

Данные для тестирования

In [96]:
x_test_seq = test['Sequences']
y_test = test['Score']

In [97]:
x_test_seq

idx
7292    [529, 69, 1138, 2, 1622, 616, 6, 1282, 1184, 5...
342     [99, 53, 88, 2, 15, 1536, 97, 2040, 63, 303, 2...
2216    [148, 1080, 8545, 23, 87, 50, 2, 202, 1801, 24...
2606    [555, 2520, 6, 631, 1618, 8, 776, 2, 559, 417,...
6572    [148, 10, 246, 1889, 2, 4, 209, 4565, 1592, 83...
                              ...                        
7754    [87, 3460, 2063, 499, 2018, 555, 4086, 1189, 3...
7023    [289, 7086, 317, 15, 838, 420, 2327, 2503, 615...
668     [2052, 54, 138, 6, 499, 1952, 882, 7695, 1082,...
5174    [112, 355, 334, 7743, 6466, 531, 9965, 1917, 3...
6535    [394, 340, 33, 4472, 5552, 485, 7411, 1230, 65...
Name: Sequences, Length: 2800, dtype: object

In [98]:
y_test

idx
7292    0
342     0
2216    0
2606    0
6572    0
       ..
7754    1
7023    0
668     0
5174    0
6535    1
Name: Score, Length: 2800, dtype: int64

## Создаем мешок слов

In [99]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.
    return results

In [100]:
x_train = vectorize_sequences(x_train_seq, max_words)

In [101]:
x_test = vectorize_sequences(x_test_seq, max_words)

In [102]:
x_train[0][:100]

array([0., 0., 2., 0., 2., 0., 0., 2., 0., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 3., 3., 1., 2., 0., 0., 0., 3., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.])

In [103]:
len(x_train[0])

10000

## Создаем модель машинного обучения

In [104]:
lr = LogisticRegression(random_state=random_state, max_iter=500)

## Обучаем модель машинного обучения

In [105]:
lr.fit(x_train, y_train)

LogisticRegression(max_iter=500, random_state=42)

## Оцениваем качество обучения на тестовом наборе данных

Определяем долю правильных ответов (accuracy) на тестовом наборе данных

In [106]:
lr.score(x_test, y_test)

0.9439285714285715

## Применяем модель для определения тональности отзыва на банк

**Позитивный отзыв**

In [107]:
positive_text = """Брал кредит в Мегабанке на автомобиль. Выдали за один день. Никаких скрытых комиссий и переплат.
У банка удобное мобильное приложение, через которое можно быстро отправить ежемесячный платеж.
Досрочное гасить начал через три месяца. Я доволен оперативностью и удобством. Огромное спасибо!
"""

Подготовка текста к обработке

In [109]:
positive_preprocessed_text = preprocess(positive_text, stops_words_ru)

In [110]:
positive_preprocessed_text

['брал',
 'кредит',
 'мегабанке',
 'автомобиль',
 'выдали',
 'день',
 'никаких',
 'скрытых',
 'комиссий',
 'переплат',
 'банка',
 'удобное',
 'мобильное',
 'приложение',
 'которое',
 'быстро',
 'отправить',
 'ежемесячный',
 'платеж',
 'досрочное',
 'гасить',
 'начал',
 'месяца',
 'доволен',
 'оперативностью',
 'удобством',
 'огромное',
 'спасибо']

In [111]:
positive_seq = text_to_sequence(positive_preprocessed_text, word_to_index)

In [112]:
positive_seq

[566,
 9,
 1678,
 440,
 10,
 81,
 3936,
 1021,
 7789,
 2,
 1568,
 3084,
 1338,
 598,
 56,
 874,
 1435,
 100,
 800,
 2645,
 560,
 93,
 453,
 6657,
 548,
 25]

In [113]:
positive_bow = vectorize_sequences([positive_seq], max_words)

In [114]:
positive_bow[0][0:100]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

Выполняем распознавание

In [115]:
result = lr.predict(positive_bow)

In [116]:
result

array([1], dtype=int64)

**Негативный отзыв**

In [117]:
negative_text = """Взял кредит в ТакСебеБанке на автомобиль. В договор включили обязательный контракт
на помощь на дороге, который мне не нужен. Узнал об этом только во время подписания договора, иначе бы отказался.
Альтернативы была страхование жизни, но мне это даже не предложили. Скорее всего, менеджер продвигает
продажи услуг этой компании в ущерб интересов клиента. Как минимум, непорядочно и непрофессионально.
У банка ужасное мобильное приложение, из-за которого с меня взяли штраф 10 тыс.руб. По требованиям
банка после покупки автомобиля в приложении нужно загрузить ПТС. Я загрузил и проверил, что ПТС в приложении есть.
Но через некоторое время ПТС из приложения пропал и с меня взяли штраф. Никому не рекомендую связываться с ТакСебеБанком.
"""

In [118]:
negative_preprocessed_text = preprocess(negative_text, stops_words_ru)
negative_seq = text_to_sequence(negative_preprocessed_text, word_to_index)
negative_bow = vectorize_sequences([negative_seq], max_words)

In [119]:
negative_bow[0][0:100]

array([0., 0., 2., 0., 1., 0., 0., 0., 0., 1., 0., 2., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [120]:
result = lr.predict(negative_bow)

In [121]:
result

array([0], dtype=int64)

In [122]:
result = lr.predict_proba(negative_bow)

In [123]:
result

array([[0.61580654, 0.38419346]])